In [ ]:
import numpy as np

file = "../data/telco/npy/sms-call-internet-mi-2013-12-01.npy"    

with open(file, "rb") as f:
    data = np.load(f)

In [ ]:
# Cols: 0-square_id, 1-timestamp, 2-country, 3-sms_in
#       4-sms_out, 5-calls_in, 6-calls_out, 7-data
data[2]

In [ ]:
data[:, 3].mean()

In [ ]:
np.nanmean(data[:, 3])

In [ ]:
np.nansum(data[:, 3:7], axis=0)

In [ ]:
import os

DIR = "../data/telco/npy"
npy_files = os.listdir(DIR)
npy_files[:4]

In [ ]:
import dask.array as da

arrays = []
for array in npy_files:
    file = os.path.join(DIR, array)
    data = np.load(file, mmap_mode="r+")
    arr = da.from_array(data, chunks=(5_000_000, 8))
    arrays.append(arr)
del data, arr

a = da.concatenate(arrays)
a

In [ ]:
%time da.to_npy_stack("../data/telco/stack", a)

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
import dask.array as da

In [ ]:
arr = da.from_npy_stack("../data/telco/stack/")
arr

In [ ]:
task = da.nanmean(arr[:, 3])
task

In [ ]:
# Direct imports:
from numpy import nanmean
from dask.array import nanmean

# Ugly 'cheating':
import numpy as np
import dask.array as np

In [ ]:
task.compute()

In [ ]:
da.nansum(arr[:, 3:7], axis=0).compute()

In [ ]:
import pandas as pd

colnames = ["square_id", "timestamp", "country_prefix",
             "sms_out", "sms_in", "call_out", "call_in", "data"]

In [ ]:
df = pd.read_csv(
        "../data/telco/txt/sms-call-internet-mi-2013-12-29.txt",
        delimiter="\t", header=None)
df.columns = colnames

In [ ]:
df = df.assign(timestamp=pd.to_datetime(df["timestamp"], unit="ms"))
df.head()

In [ ]:
df = df.assign(hour=df["timestamp"].dt.hour)
df = df.drop(["square_id", "timestamp", "country_prefix", "data"], axis=1)

hour_sums = df.groupby("hour").sum()

hour_sums.head()

In [ ]:
%matplotlib inline
hour_sums.plot()

In [ ]:
import dask.dataframe as dd

df = dd.read_csv(
        "../data/telco/txt/sms-call-internet-mi-*",
        delimiter="\t", header=None)
df.columns = colnames
df

In [ ]:
df = df.assign(timestamp=dd.to_datetime(df["timestamp"], unit="ms"))
df = df.assign(hour=df["timestamp"].dt.hour)
df = df.drop(["square_id", "timestamp", "country_prefix", "data"], axis=1)

In [ ]:
task = df.groupby("hour").sum()
task

In [ ]:
result = task.compute()

In [ ]:
%matplotlib inline
result.plot()